In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torch
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import timm
import csv

import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt

from dataloader import DigitDataset
from utils import reshape, drawimg, tensor_to_pltimg, multidrawimg, train, test
#Test

In [2]:
device = (torch.device('cuda') if torch.cuda.is_available()
         else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


### Data Loader

In [3]:
trainfile = pd.read_csv('digit-recognizer/train.csv')
trainimg = [(np.reshape(row[1].values[1:],(28,28)),row[1].values[0]) for row in trainfile.iterrows()]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5,], [0.5,]),
    transforms.RandomAdjustSharpness(10)]
    
    )
batch_size=64
ds_train = DigitDataset('digit-recognizer/train.csv', transform)

dl_train = DataLoader(dataset=ds_train, batch_size=batch_size)


train_dataset, valid_dataset = train_test_split(ds_train, test_size=0.25, random_state=123, shuffle=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=0, shuffle=False)

imgs, labels = next(iter(dl_train))

imgs.shape, labels.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

### Model

In [4]:
model = timm.create_model('resnet34')
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model = model.to('cuda')
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

21791400

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [6]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer,device=device)
    test(valid_loader, model, loss_fn, device=device)
print("Done!")

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Epoch 1
-------------------------------
loss: 6.958308  [    0/31500]
loss: 5.376609  [ 6400/31500]
loss: 4.638804  [12800/31500]
loss: 3.577127  [19200/31500]
loss: 3.010763  [25600/31500]
Test Error: 
 Accuracy: 47.3%, Avg loss: 2.665367 

Epoch 2
-------------------------------
loss: 2.533767  [    0/31500]
loss: 1.650608  [ 6400/31500]
loss: 1.239842  [12800/31500]
loss: 1.395901  [19200/31500]
loss: 1.547235  [25600/31500]
Test Error: 
 Accuracy: 47.9%, Avg loss: 1.559759 

Epoch 3
-------------------------------
loss: 1.403993  [    0/31500]
loss: 1.336676  [ 6400/31500]
loss: 1.225676  [12800/31500]
loss: 1.208741  [19200/31500]
loss: 1.142883  [25600/31500]
Test Error: 
 Accuracy: 48.4%, Avg loss: 1.497888 

Epoch 4
-------------------------------
loss: 1.652403  [    0/31500]
loss: 1.178775  [ 6400/31500]
loss: 1.396020  [12800/31500]
loss: 1.261415  [19200/31500]
loss: 1.053271  [25600/31500]
Test Error: 
 Accuracy: 49.4%, Avg loss: 1.397456 

Epoch 5
------------------------

In [3]:
batch_size = 64
loss_fn = nn.CrossEntropyLoss()
ds_test = DigitDataset('digit-recognizer/test.csv', transforms.ToTensor())
test_loader = DataLoader(dataset=ds_test, batch_size=batch_size, shuffle=False)

model = timm.create_model('resnet34')
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model = model.to('cuda')
model.load_state_dict(torch.load("model.pth"))
test(test_loader, model, loss_fn, device=device, valid=False)

24007
58002
43007
5007
36007
32007
45007
13007
5007
29002
19007
26007
50007
25003
13007
22007
60007
51007
25007
26007
38007
40007
23007
14007
37007
42007
25007
27007
54007
7007
35007
15007
14007
11007
14007
52002
9007
9007
28007
60007
25007
44007
21007
23007
34007
36005
55002
30003
1007
20002
50007
55007
62007
47007
11007
8007
56003
62007
49007
21007
32002
29007
31007
8007
20007
8007
54007
37007
6007
31007
6007
26007
16007
53007
56007
62003
57007
42007
4003
14007
33007
59007
60007
17007
55007
43002
19007
5007
23003
54007
30007
38007
47007
51007
39007
26007
18007
63007
8007
21007
6007
18007
58007
47007
52007
5007
37007
44007
27007
36007
55007
54007
52007
20007
36007
43007
55007
56007
16007
56007
8007
38007
61007
47007
26007
47007
51007
62007
29007
28007
23002
7002
16007
50007
20007
44007
26007
49007
17007
47007
52007
55007
42007
21007
54007
23007
6007
31007
47007
54007
24007
41007
32007
42007
22007
42007
41007
55007
51007
8007
53007
45007
52007
47007
28007
1002
18007
8002
51007
33007
56